In [1]:
# Import pandas library using an alias
import pandas as pd
# library to handle data in a vectorized manner
import numpy as np

# library to handle JSON files
# import json
# library to handle requests
# import requests
# tranform JSON file into a pandas dataframe
# from pandas.io.json import json_normalize

# import geocoder
import geocoder
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# map rendering library
import folium

In [4]:
library_df = pd.read_excel('ChicagoLibraries.xlsx')
library_df.head()

,name_,address,city,state,zip,phone,hours_of_operation,latitude,longitude,url
0,Jefferson Park,5363 W. Lawrence Ave.,Chicago,IL,60630,Closed for Construction,Not listed,41.967597,-87.761554,https://www.chipublib.org/locations/38/
1,Merlo,644 W. Belmont Ave.,Chicago,IL,60657,Closed for Construction,Not listed,41.940084,-87.646143,https://www.chipublib.org/locations/51/
2,Douglass,3353 W. 13th St.,Chicago,IL,60623,Closed for Construction,Not listed,41.864501,-87.709685,https://www.chipublib.org/locations/25/
3,Albany Park,3401 W. Foster Ave.,Chicago,IL,60625,(773) 539-5450,"Sun., Closed; Mon. & Wed., 10-6; Tue. & Thu., ...",41.975456,-87.714090,https://www.chipublib.org/locations/3/
4,Altgeld,13281 S. Corliss Ave.,Chicago,IL,60827,(312) 747-3270,"Sun., Closed; Mon. & Wed., Noon-8; Tue. & Thu....",41.654730,-87.602230,https://www.chipublib.org/locations/4/


In [9]:
library_df.columns

Index(['name_', 'address', 'city', 'state', 'zip', 'phone',
       'hours_of_operation', 'latitude', 'longitude', 'url'],
      dtype='object')

In [11]:
# Get coordinates for Chicago
address = 'Chicago, IL'
geolocator = Nominatim(user_agent='chi_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Chicago are {}, {}".format(latitude, longitude))

The geographical coordinates of Chicago are 41.8755616, -87.6244212


In [12]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude])

In [13]:
# Add markers to the map
for lat, lon, name in zip(library_df['latitude'],
                          library_df['longitude'],
                          library_df['name_']):
    label = folium.Popup(str(name), parse_html=True)
    folium.CircleMarker(
       [lat, lon],
       radius=5,
       popup=label,
       fill=True,
       fill_opacity=0.7).add_to(chicago_map)
    
# Display the map
chicago_map